In [1]:
# dependencies
from pathlib import Path
import subprocess
import yaml

In [2]:
# support methods
def read_textlike(fname, concat_fname=False):
    if not concat_fname:
        with open(fname, 'r') as f:
            lines = [line for line in f.readlines() if 'todo' in line.lower()]
    else:
        with open(fname, 'r') as f:
            lines = [line+f" {fname}" for line in f.readlines() if 'todo' in line.lower()]
    return lines


def exists_or_mkdir(path):
    if not Path(path).exists():
        print(f"{path} does not exist. Adding now...")
        subprocess.call(['mkdir', path])
        subprocess.call(['touch', f'{path}/todo.done'])
        print("added path.")
    return 1


def read_yaml(fname):
    with open(fname, 'r') as f_handle:
        out = yaml.safe_load(f_handle)
    return out


def track_tasks(fname, data):
    with open(fname, 'a') as f:
        yaml.dump(data, f, default_flow_style=False)
        f.close()
    print(f'{fname} updated successfully, {len(data)} item(s) added.')
    return 1


def reformat_line(line):
    if line == '':
        return None
    idx = line.lower().find('todo')
    if idx < 0:
        return None
    form = line.lower()[idx+5:].strip()
    return form


# NOTE: this approach abandons text between "todo" and "("
def get_primarytag(line):
    if (line is None) | (line == ''):
        return None
    l = line.find('(')
    if l < 0:
        return 'untagged'
    r = line.find(')')
    tag = line[l:r+1].strip()
    return tag


def get_task(line, tag):
    if tag == 'untagged':
        return line
    idx = line.find(tag)
    rem = line[idx+len(tag)+1:]
    return rem


def clean_tag(tag):
    return tag.replace('(', '').replace(')', '').replace(' ', '_')


def process_lines(lines):
    out = {}
    for line in lines:
        print(f'line:\t{line[:-1]}')
        info = reformat_line(line)
        print(f'form:\t{info}')
        tag = get_primarytag(info)
        print(f'tag:\t{tag}')
        task = get_task(info, tag)
        print(f'task:\t{task}\n')
        tag = clean_tag(tag)
        if tag not in out:
            out[tag] = [task]
        else:
            out[tag].append(task)
    return out


def drop_curr_tasks(fname, task_list):
    curr = set(read_yaml(fname))
    inc = set(task_list)
    keep = inc - curr
    return list(keep)


def write_tasks(write_loc, task_dict):
    exists_or_mkdir(write_loc)
    if write_loc[-1] != "/":
        write_loc += "/"
    for tag, task_list in task_dict.items():
        tag_dir = write_loc + tag
        exists_or_mkdir(f"{tag_dir}")
        fname = f"{tag_dir}/todo.yml"
        if Path(fname).exists():
            print(f"checking {len(task_list)} found tasks against those in {fname}.")
            new_tasks = drop_curr_tasks(fname, task_list)
            print(f"{len(task_list) - len(new_tasks)} duplicate tasks dropped. {len(new_tasks)} waiting to be added.")
        else:
            new_tasks = task_list
        if len(new_tasks) > 0:
            track_tasks(fname, new_tasks)
        else:
            print("no new tasks found.")
    return 1

In [3]:
# main
arg_hand = "../hand/code-review.md"
arg_output = "../output/"

instrs = read_textlike(arg_hand)
task_dict = process_lines(instrs)

write_tasks(arg_output, task_dict)

line:	- TODO: add the code review prompts to a training doc
form:	add the code review prompts to a training doc
tag:	untagged
task:	add the code review prompts to a training doc

line:	- TODO: (longterm) practice R
form:	(longterm) practice r
tag:	(longterm)
task:	practice r

line:	- TODO: (project 2) import Makefile targets need better names
form:	(project 2) import makefile targets need better names
tag:	(project 2)
task:	import makefile targets need better names

line:	- TODO: (project 2) import.R initial_asserts() needs improvement
form:	(project 2) import.r initial_asserts() needs improvement
tag:	(project 2)
task:	import.r initial_asserts() needs improvement

line:	- TODO: (project 2) import.R apply TS advice about short functions
form:	(project 2) import.r apply ts advice about short functions
tag:	(project 2)
task:	import.r apply ts advice about short functions

line:	- TODO: (project 2) sample_state.R apply TS advice about short functions
form:	(project 2) sample_state.r apply

1

In [4]:
write_tasks(arg_output, task_dict)

checking 1 found tasks against those in ../output/untagged/todo.yml.
1 duplicate tasks dropped. 0 waiting to be added.
no new tasks found.
checking 1 found tasks against those in ../output/longterm/todo.yml.
1 duplicate tasks dropped. 0 waiting to be added.
no new tasks found.
checking 16 found tasks against those in ../output/project_2/todo.yml.
16 duplicate tasks dropped. 0 waiting to be added.
no new tasks found.
checking 1 found tasks against those in ../output/reading/todo.yml.
1 duplicate tasks dropped. 0 waiting to be added.
no new tasks found.


1